# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
E. Rohr  ->  E. Rohr  |  ['E. Rohr']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
M. Prunier  ->  M. Prunier  |  ['M. Prunier']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
D. Chatzigiannakis  ->  D. Chatzigiannakis  |  ['D. Chatzigiannakis']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Liu  ->  J. Liu  |  ['J. Liu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 84 new papers today
          12 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/12 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.01953
extracting tarball to tmp_2503.01953...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']


Found 70 bibliographic references in tmp_2503.01953/manuscript.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.01960


...

extracting tarball to tmp_2503.02257...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.02520


extracting tarball to tmp_2503.02520... done.
Retrieving document from  https://arxiv.org/e-print/2503.02884


extracting tarball to tmp_2503.02884...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.01953-b31b1b.svg)](https://arxiv.org/abs/2503.01953) | **A merging pair of massive quiescent galaxies at $z=3.44$ in the Cosmic Vine**  |
|| K. Ito, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>L. Xie</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *8 pages, 5 figures, 1 table + Appendix. Accepted for publication in A&A on Feb 28, 2025. Spectra and photometry used in this paper are available at this https URL , See Valentino et al. (2025) on arXiv today for another result from the JWST "DeepDive" program*|
|**Abstract**|            We report the spectroscopic confirmation of a merging pair of massive quiescent galaxies at $z=3.44$. Using JWST observations, we confirm that the two galaxies lie at a projected separation of 4.5 kpc with a velocity offset of $\sim 680\, {\rm km\, s^{-1}}\ (\delta_z \sim 0.01)$. The pair resides in the core of a known rich overdensity of galaxies, dubbed the "Cosmic Vine". For both pair members, modeling of the Spectral Energy Distributions and faint rest-frame optical emission lines indicate high stellar masses ($\log{(M_\star/M_\odot)}\sim10.9$) and suppressed star formation ($\log{\rm (sSFR/yr^{-1})}<-10$), more than an order of magnitude below the level of the star formation main sequence at this redshift. We then explore the Illustris-TNG simulation and the GAEA and SHARK semi-analytical models to examine whether they produce a pair of massive quiescent galaxies akin to that of the Cosmic Vine. While all models produce close pairs of massive quiescent galaxies at $2<z<4$ with comparable separations and velocity offsets, their predicted number densities are $10-80$ times lower than our observational constraint. This discrepancy cannot be fully explained by coarse time sampling in these models or the general challenge of forming early massive quiescent galaxies in simulations. Given that $>90\%$ of simulated pairs in the models that we analyzed merge by $z=0$, our findings suggest that our observed pair will likely coalesce into a single massive galaxy. The merger, occurring in the dense core of a large-scale structure, might represent a critical event in the formation of a brightest cluster galaxy and the morphological transformation of high-redshift disky quiescent galaxies into early-type ellipticals.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.01960-b31b1b.svg)](https://arxiv.org/abs/2503.01960) | **The abundance and origin of cool gas in galaxy clusters in the TNG-Cluster simulation**  |
|| M. Staffehl, et al. -- incl., <mark>E. Rohr</mark>, <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *The TNG-Cluster simulation is now fully publicly released at this https URL (see also science results today from Prunier+, Chatzigiannakis+, and Lehle+ today)*|
|**Abstract**|            In addition to the hot intracluster medium, massive galaxy clusters host complex, multi-phase gaseous halos. In this work, we quantify the abundance, spatial distribution, and origin of the cool T < 10^4.5 K gas within clusters. To do so, we combine the TNG-Cluster and TNG300 cosmological magnetohydrodynamical simulations, yielding a sample of 632 simulated galaxy clusters at z=0 with masses M_200c ~ 10^14-15.4 solar masses. We find that cool gas is present in every cluster at z=0, although it constitutes only a small fraction of the total gas mass within twice the virial radius, ranging from ~10^-4 to a few per cent. The majority of cool gas resides in the cluster outskirts in infalling satellites and other halos. More rarely, cool gas can also be present in the central regions of clusters. More massive halos contain larger amounts (but not fractions) of cool gas ~10^10-12 solar masses, and we identify correlations between cluster cool gas fraction and several global halo and galaxy properties at fixed halo mass. Using Monte-Carlo Lagrangian tracer particles, we then track the origin of cool gas in present-day clusters. We find that the primary source is recent accretion at z < 0.1, predominantly in the form of pre-cooled gas carried by infalling satellite galaxies and other halos. However, in-situ cooling of the hot intracluster medium gas accreted at earlier epochs also contributes, especially in present-day cool-core clusters.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.01965-b31b1b.svg)](https://arxiv.org/abs/2503.01965) | **X-ray cavities in TNG-Cluster: a direct comparison to observations**  |
|| <mark>M. Prunier</mark>, <mark>A. Pillepich</mark>, J. Hlavacek-Larrondo, D. Nelson |
|*Appeared on*| *2025-03-05*|
|*Comments*| *Submitted to MNRAS. The data of the TNG-Cluster simulation is now fully public and accessible at this https URL. See also science results by Chatzigiannakis+, Lehle+ and Staffehl+ on astro-ph today*|
|**Abstract**|            The TNG-Cluster magnetohydrodynamic cosmological simulations, produce a diverse population of X-ray cavities in the intracluster medium (ICM) of simulated galaxy clusters. These arise from episodic, high velocity, kinetic energy injections from the central active supermassive black hole (AGN, SMBH). Here, we present the first comprehensive comparative analysis of X-ray cavities in TNG-Cluster with observational data. First, we select a volume-limited sample of 35 real clusters ($z \leq 0.071$, M$_\text{500c}$ = 10$^{14-14.8}$ M$_\odot$) observed with the Chandra X-ray Observatory, identify 3 analogs for each in TNG-Cluster (total of 105) and generate mock Chandra images using same exposure times as their observed counterparts. We identify X-ray cavities and measure their properties in both datasets using identical techniques, ensuring a direct, apples-to-apples comparison. Our analysis reveals that both samples have a similar fraction of X-ray cavities (35-43 per cent). They exhibit comparable sizes and morphologies, although the sizes of simulated X-ray cavities still attached to the SMBH are somewhat larger in TNG-Cluster -- a scarcity at $< 10$ kpc. The area of TNG X-ray cavities increases as they rise in the ICM, consistent with the trend seen in the observational sample. The cavity powers, estimated using observational techniques, show good agreement between the two samples (10$^{42-45}$ erg.s$^{-1}$), suggesting that X-ray cavities in the simulation are an important heating mechanism in cluster cores. Overall, the rather simple AGN feedback model of TNG, with no model choices made to reproduce X-ray morphological features, and without cosmic rays, creates a quantitatively realistic population of X-ray cavities at cluster scales.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.01969-b31b1b.svg)](https://arxiv.org/abs/2503.01969) | **What Drives Cluster Cool-Core Transformations? A Population Level Analysis of TNG-Cluster**  |
|| K. Lehle, D. Nelson, <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *The data of the TNG-Cluster simulation is now fully public and accessible at this https URL. See also science results by Chatzigiannakis+, Prunier+ and Staffehl+ on astro-ph today. Submitted to A&A*|
|**Abstract**|            In this study, we examine the frequency and physical drivers of transformations from cool-core (CC) to non-cool-core (NCC) clusters, and vice versa, in a sample of 352 massive galaxy clusters (M_vir = 10^14-15.3 M_sun) from the TNG-Cluster magnetohydrodynamical cosmological simulation of galaxies. By identifying transformations based on the evolution of central entropy and focusing on z<2.5, we find that clusters frequently undergo such events, depending on their assembly and supermassive black hole histories. On average, clusters experience 2 to 3 transformations. Transformations can occur in both directions and can be temporary, but those to higher entropy cores, i.e. in the direction from CC to NCC states, are the vast majority. CC phases are shorter than NCC phases, and thus overall the TNG-Cluster population forms with low-entropy cores and moves towards NCC states with time. We study the role that mergers play in driving transformations, and find that mergers within ~1Gyr prior to a transformation toward higher (but not lower) entropy cores occur statistically more often than in a random control sample. Most importantly, we find examples of mergers associated with CC disruption regardless of their mass ratio or angular momentum. However, past merger activity is not a good predictor for z=0 CC status, at least based on core entropy, even though clusters undergoing more mergers eventually have the highest core entropy values at z=0. We consider the interplay between AGN feedback and evolving cluster core thermodynamics. We find that core transformations are accompanied by an increase in AGN activity, whereby frequent and repeated (kinetic) energy injections from the central SMBHs can produce a collective, long-term impact on central entropy, ultimately heating cluster cores. Whether such fast-paced periods of AGN activity are triggered by mergers is plausible, but not necessary.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.02257-b31b1b.svg)](https://arxiv.org/abs/2503.02257) | **SN 2021hpr: A Normal Type Ia Supernova Showing Excess Emission in the Early Rising Phase**  |
|| A. Iskandar, et al. -- incl., <mark>J. Liu</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *27 pages, 15 figures, Accepted for publication in ApJ*|
|**Abstract**|            We present extensive optical observations of a nearby Type Ia supernova (SN Ia), SN 2021hpr, located in the spiral galaxy NGC 3147 at a distance of $\sim$ 45 Mpc. Our observations cover a phase within $\sim 1-2$ days to $\sim 290$ days after the explosion. SN 2021hpr is found to be a spectroscopically normal SN Ia, with an absolute B-band peak magnitude of $M_{max}(B) \approx -19.16 \pm 0.14$ mag and a post-peak decline rate of $\Delta m_{15}(B)= 1.00 \pm 0.01 $ mag. Early-time light curves showed a $\sim 7.0 \%$ excess emission compared to a homogeneously expanding fireball model, likely due to SN ejecta interacting with a companion or immediate circumstellar matter. The optical spectra of SN 2021hpr are overall similar to those of normal SNe Ia, but characterized by prominent detached high-velocity features (HVFs) of Si {\sc ii} and Ca {\sc ii} in the early phase. After examining a small sample of well-observed normal SNe Ia, we find that the HVFs are likely common for the subgroup with early-excess emission. The association of early bump feature with the HVFs could be attributed to density or abundance enhancement at the outer layer of the exploding star, likely as a result of interactions with companion$/$CSM or experiencing more complete burning. Nevertheless, the redshifted Fe {\sc ii} and Ni {\sc ii} lines in the nebular-phase spectra of SN 2021hpr, contrary to the blueshift trend seen in other SNe Ia showing early bump features, indicate its peculiarity in the explosion that remains to be understood.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.01983-b31b1b.svg)](https://arxiv.org/abs/2503.01983) | **The temperature and metallicity distributions of the ICM: insights with TNG-Cluster for XRISM-like observations**  |
|| <mark>D. Chatzigiannakis</mark>, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *The data of the TNG-Cluster simulation is now fully public and accessible at this http URL . See also science results by Prunier+, Lehle+ and Staffehl+ on astro-ph today*|
|**Abstract**|            The new era of high-resolution X-ray spectroscopy will significantly improve our understanding of the intra-cluster medium (ICM) by providing precise constraints on its underlying physical properties. However, spectral fitting requires reasonable assumptions on the thermal and chemical distributions of the gas. We use the output of TNG-Cluster, the newest addition to the IllustrisTNG suite of cosmological magnetohydrodynamical simulations, to provide theoretical expectations for the multi-phase nature of the ICM across hundreds of z=0 clusters (M$_{500c}$ = 10$^{14.0-15.3}$ M$_\odot$) based upon a realistic model for galaxy formation and evolution. We create and analyse, in an observer-like manner, end-to-end XRISM/Resolve mock observations towards cluster centres. We then systematically compare the intrinsic properties of the simulated gas with the inferred ones from spectral fitting via a variety of commonly used spectral-emission models. Our analysis suggests that models with a distribution of temperatures, such as bvlognorm and bvgadem, better describe the complex thermal structure of the ICM, as predicted by TNG-Cluster, but incur biases of 0.5-2 keV (16th-84th percentiles). The 1T bvapec is too simplistic for the predicted broad temperature distributions, while a 2T double bvapec model systematically fails to capture the input temperature structure. However, all spectral emission models systematically underestimate the Fe abundance of the central ICM by ~0.1 Solar (~ 20 per cent) primarily due to projection effects. Selecting only strong cool core clusters leads to minor improvements on inference quality, removing the majority of outliers but maintaining similar overall biases and cluster-to-cluster scatter.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.02059-b31b1b.svg)](https://arxiv.org/abs/2503.02059) | **A confirmed deficit of hot and cold dust emission in the most luminous Little Red Dots**  |
|| D. J. Setton, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *16 pages, 5 figures, 3 tables. Submitted to ApJ Letters. Comments welcome!*|
|**Abstract**|            Luminous broad H$\alpha$ emission and red rest-optical SEDs are the hallmark of compact Little Red Dots (LRDs), implying highly attenuated dusty starbursts and/or obscured active galactic nuclei. However, the lack of observed FIR emission has proved difficult to reconcile with the implied attenuated luminosity in these models. Here, we utilize deep new ALMA imaging, new and existing JWST/MIRI imaging, and archival Spitzer/Herschel imaging of two of the rest-optically brightest LRDs ($z=3.1$ and $z=4.47$) to place the strongest constraints on the IR luminosity in LRDs to date. The detections at $\lambda_\mathrm{rest}=1-4 \ \mu$m imply flat slopes in the rest-IR, ruling out a contribution from hot ($T\gtrsim500$ K) dust. Similarly, FIR non-detections rule out any appreciable cold ($T\lesssim75$ K) dust component. Assuming energy balance, these observations are inconsistent with the typical FIR dust emission of dusty starbursts and quasar torii, which usually show a mixture of cold and hot dust. Additionally, our [$\mathrm{C}_{II}$] non-detections rule out typical dusty starbursts. We compute empirical maximum IR SEDs and find that both LRDs must have $\log(L_\mathrm{IR}/L_\odot) \lesssim 12.2$ at the $3\sigma$ level. These limits are in tension with the predictions of rest-optical spectrophotometric fits, be they galaxy only, AGN only, or composite. It is unlikely that LRDs are highly dust-reddened intrinsically blue sources with a dust temperature distribution that conspires to avoid current observing facilities. Rather, we favor an intrinsically redder LRD SED model that alleviates the need for strong dust attenuation.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.02227-b31b1b.svg)](https://arxiv.org/abs/2503.02227) | **Deriving Stellar Properties, Distances, and Reddenings using Photometry and Astrometry with BRUTUS**  |
|| J. S. Speagle, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *33 pages, 17 figures main text (+23 pages, +10 figures in appendices). To be re-submitted to AAS Journals. Package documentation to be added in coming weeks (as of 2025-03-03)*|
|**Abstract**|            We present brutus, an open source Python package for quickly deriving stellar properties, distances, and reddenings to stars based on grids of stellar models constrained by photometric and astrometric data. We outline the statistical framework for deriving these quantities, its implementation, and various Galactic priors over the 3-D distribution of stars, stellar properties, and dust extinction (including $R_V$ variation). We establish a procedure to empirically calibrate MIST v1.2 isochrones by using open clusters to derive corrections to the effective temperatures and radii of the isochrones, which reduces systematic errors on the lower main sequence. We also describe and apply a method to estimate photometric offsets between stellar models and observed data using nearby, low-reddening field stars. We perform a series of tests on mock and real data to examine parameter recovery with MIST under different modeling assumptions, illustrating that brutus is able to recover distances and other stellar properties using optical to near-infrared photometry and astrometry. The code is publicly available at this https URL.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 41</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.02200-b31b1b.svg)](https://arxiv.org/abs/2503.02200) | **Mapping the Milky Way in 5-D with 170 Million Stars**  |
|| J. S. Speagle, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-03-05*|
|*Comments*| *Published in ApJ in 2024. 31 pages, 17 figures*|
|**Abstract**|            We present "augustus", a catalog of distance, extinction, and stellar parameter estimates to 170 million stars from $14\,{\rm mag} < r < 20\,{\rm mag}$ and with $|b| > 10^\circ$ drawing on a combination of optical to near-IR photometry from Pan-STARRS, 2MASS, UKIDSS, and unWISE along with parallax measurements from \textit{Gaia} DR2 and 3-D dust extinction maps. After applying quality cuts, we find 125 million objects have "high-quality" posteriors with statistical distance uncertainties of $\lesssim 10\%$ for objects with well-constrained stellar types. This is a substantial improvement over distance estimates derived from Gaia parallaxes alone and in line with results from previous work. We find the fits are able to accurately reproduce the de-reddened Gaia color-magnitude diagram, which serves as a useful consistency check of our results. We show that we are able to clearly detect large, kinematically-coherent substructures in our data relative to the input priors, including the Monoceros Ring and the Sagittarius stream, attesting to the quality of the catalog. Our results are publicly available at doi:https://doi.org/10.7910/DVN/WYMSXV. An accompanying interactive visualization can be found at this http URL.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 44</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.01953/./spectrum_summary.png', 'tmp_2503.01953/./Numberdensity_QGpair.png', 'tmp_2503.01953/./QGpair_image.png']
copying  tmp_2503.01953/./spectrum_summary.png to _build/html/
copying  tmp_2503.01953/./Numberdensity_QGpair.png to _build/html/
copying  tmp_2503.01953/./QGpair_image.png to _build/html/
exported in  _build/html/2503.01953.md
    + _build/html/tmp_2503.01953/./spectrum_summary.png
    + _build/html/tmp_2503.01953/./Numberdensity_QGpair.png
    + _build/html/tmp_2503.01953/./QGpair_image.png
found figures ['tmp_2503.01960/./img/tracer_history_final_plots_tracer_fraction_TNG_Cluster_within_2Rvir.png', 'tmp_2503.01960/./img/tracer_history_crossing_times_TNG_Cluster_distribution_unweighted.png', 'tmp_2503.01960/./img/tracer_history_crossing_times_TNG_Cluster_mean_vs_mass.png', 'tmp_2503.01960/./img/tracer_history_time_development_distance_TNG_Cluster_2dhist_mean.png', 'tmp_2503.01960/./img/tracer_history_time_development_temperature_TNG_Cluster_mean.pn

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\oiii}{[O {\footnotesize III}]}$
$\newcommand{\oii}{[O {\footnotesize II}]}$
$\newcommand{\nii}{[N {\footnotesize II}]}$
$\newcommand{\sii}{[S {\footnotesize II}]}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\myr}{M_{\odot} yr^{-1}}$
$\newcommand{\hii}{H {\footnotesize II}}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A merging pair of massive quiescent galaxies at $z=3.44$ in the Cosmic Vine

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.01953-b31b1b.svg)](https://arxiv.org/abs/2503.01953)<mark>Appeared on: 2025-03-05</mark> -  _8 pages, 5 figures, 1 table + Appendix. Accepted for publication in A&A on Feb 28, 2025. Spectra and photometry used in this paper are available at this https URL , See Valentino et al. (2025) on arXiv today for another result from the JWST "DeepDive" program_

</div>
<div id="authors">

K. Ito, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>L. Xie</mark>

</div>
<div id="abstract">

**Abstract:** We report the spectroscopic confirmation of a merging pair of massive quiescent galaxies at $z=3.44$ . Using $* JWST*$ observations, we confirm that the two galaxies lie at a projected separation of 4.5 kpc with a velocity offset of $\sim 680$ $\kms$ ( $\delta_z \sim 0.01$ ). The pair resides in the core of a known rich overdensity of galaxies, dubbed the "Cosmic Vine". For both pair members, modeling of the Spectral Energy Distributions and faint rest-frame optical emission lines indicate high stellar masses ( $\log{(M_\star/M_\odot)}\sim10.9$ ) and suppressed star formation ( $\log{\rm (sSFR/yr^{-1})}<-10$ ), more than an order of magnitude below the level of the star formation main sequence at this redshift. We then explore the Illustris-TNG simulation and the GAEA and SHARK semi-analytical models to examine whether they produce a pair of massive quiescent galaxies akin to that of the Cosmic Vine. While all models produce close pairs of massive quiescent galaxies at $2<z<4$ with comparable separations and velocity offsets, their predicted number densities are 10–80 times lower than our observational constraint. This discrepancy cannot be fully explained by coarse time sampling in these models or the general challenge of forming early massive quiescent galaxies in simulations. Given that $>90$ \% of simulated pairs in the models that we analyzed merge by $z=0$ , our findings suggest that our observed pair will likely coalesce into a single massive galaxy. The merger, occurring in the dense core of a large-scale structure, might represent a critical event in the formation of a brightest cluster galaxy and the morphological transformation of high-redshift disky quiescent galaxies into early-type ellipticals.

</div>

<div id="div_fig1">

<img src="tmp_2503.01953/./spectrum_summary.png" alt="Fig4" width="100%"/>

**Figure 4. -** Summary of spectra of two quiescent galaxies in the Cosmic Vine. The first two rows represent the spectrum of the 61168, and the latter two represent the 61167. For each galaxy, the upper panel shows the observed NIRSpec medium-resolution spectrum (black line), photometry (blue squares), and the best fit in SED fitting (red line). The gray hatch indicates the wavelength masked in the SED fitting. The bottom left panel shows the continuum-subtracted spectrum around the $\hb$ emission line. In addition to the observed spectrum (black) and best-fit spectrum (red), the narrow and broad components of the best-fit are shown in blue and green lines, respectively. The bottom right panel shows that around the $\ha$ emission line. The meanings of the lines are identical to those of the bottom left panel. The difference between the observed spectrum and the best-fit model is shown at the bottom of each panel. (*fig:linefit*)

</div>
<div id="div_fig2">

<img src="tmp_2503.01953/./Numberdensity_QGpair.png" alt="Fig3" width="100%"/>

**Figure 3. -** The number density of pairs of quiescent galaxies at each snapshot. Blue circles, green triangles, and yellow inverse triangles correspond to those of the TNG-300, SHARK, and GAEA, respectively. Their error bar is determined based on the Poisson noise. The red star shows the observationally estimated number density from the CEERS survey volume. Its error bar along the x-axis is $\pm 0.5  {\rm Gyr}$, which is the time used in deriving the number density.  (*fig:rho_QGpair*)

</div>
<div id="div_fig3">

<img src="tmp_2503.01953/./QGpair_image.png" alt="Fig1" width="100%"/>

**Figure 1. -** JWST/NIRCam color image of the pair of quiescent galaxies reported in this paper. F115W+F150W, F200W+F277W, and F356W+F444W images from the CEERS survey are used for blue, green, and red, respectively. This image is generated with  {\sc Trilogy}\protect$\footnote$mark. The point spread function of the blue and green images is matched to that of the F444W image. The white rectangles correspond to the open MSA shutter position. (*fig:image*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.01953"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\LambdaCDM}{\Lambda\rm CDM}$
$\newcommand{\metalline}[2]{#1 \textsc{#2}}$
$\newcommand{\nth}[1]{#1\textsuperscript{th}}$
$\newcommand{\milan}[1]{\textcolor{purple}{[Milan] #1}}$
$\newcommand{\reza}[1]{\color{magenta}#1\color{black}}$
$\newcommand{\rezac}[1]{\color{magenta}\textit{[#1]}\color{black}}$
$\newcommand{\rezas}[1]{\color{magenta}\sout{#1} \color{black}}$
$\newcommand{\er}[1]{{\color{teal}#1}}$</div>



<div id="title">

# The abundance and origin of cool gas in galaxy clusters \\in the TNG-Cluster simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.01960-b31b1b.svg)](https://arxiv.org/abs/2503.01960)<mark>Appeared on: 2025-03-05</mark> -  _The TNG-Cluster simulation is now fully publicly released at this https URL (see also science results today from Prunier+, Chatzigiannakis+, and Lehle+ today)_

</div>
<div id="authors">

M. Staffehl, et al. -- incl., <mark>E. Rohr</mark>, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** In addition to the hot intracluster medium, massive galaxy clusters host complex, multi-phase gaseous halos. In this work, we quantify the abundance, spatial distribution, and origin of the cool ( $T\leq10^{4.5} \rm{K}$ ) gas within clusters. To do so, we combine the TNG-Cluster and TNG300 cosmological magnetohydrodynamical simulations, yielding a sample of 632 simulated galaxy clusters at $z=0$ with masses $M_{200c} \sim 10^{14-15.4} \rm{M_\odot}$ . We find that cool gas is present in every cluster at $z=0$ , although it constitutes only a small fraction of the total gas mass within twice the virial radius, ranging from $\sim 10^{-4}$ to a few per cent. The majority of cool gas resides in the cluster outskirts in infalling satellites and other halos. More rarely, cool gas can also be present in the central regions of clusters. More massive halos contain larger amounts (but not fractions) of cool gas ( $\sim 10^{10-12} \rm{M_\odot}$ ), and we identify correlations between cluster cool gas fraction and several global halo and galaxy properties at fixed halo mass. Using Monte-Carlo Lagrangian tracer particles, we then track the origin of cool gas in present-day clusters. We find that the primary source is recent accretion at $z \lesssim 0.1$ , predominantly in the form of pre-cooled gas carried by infalling satellite galaxies and other halos. However, in-situ cooling of the hot intracluster medium gas accreted at earlier epochs also contributes, especially in present-day cool-core clusters.

</div>

<div id="div_fig1">

<img src="tmp_2503.01960/./img/tracer_history_final_plots_tracer_fraction_TNG_Cluster_within_2Rvir.png" alt="Fig10.1" width="25%"/><img src="tmp_2503.01960/./img/tracer_history_crossing_times_TNG_Cluster_distribution_unweighted.png" alt="Fig10.2" width="25%"/><img src="tmp_2503.01960/./img/tracer_history_crossing_times_TNG_Cluster_mean_vs_mass.png" alt="Fig10.3" width="25%"/><img src="tmp_2503.01960/./img/tracer_history_time_development_distance_TNG_Cluster_2dhist_mean.png" alt="Fig10.4" width="25%"/>

**Figure 10. -** Evolution of the spatial distribution of tracers that contribute to $z=0$ cool gas in clusters, from TNG-Cluster. In practice, here we follow those tracers that at $z=0$ are associated with gas cells of temperatures below $10^{4.5} \rm K$ and within $2R_{200c}$ of their host cluster. Top left: Fraction of selected tracers within $2R_{200c}$ as a function of redshift. The virial radius $R_{200c}$ develops with redshift and is that of the cluster progenitor which eventually hosts the cool gas at redshift $z = 0$. Top right: Mean distribution of crossing redshifts, defined as the first (green) or last (purple) redshift at which the distance from a tracer to the progenitor of its $z = 0$ host cluster decreases below $2R_{200c}$. The solid (dashed) vertical line shows the mean (median) crossing redshift. Bottom left: Mean $2R_{200c}$ crossing redshift of every cluster in TNG-Cluster versus the $z = 0$ mass of the cluster. The individual points are colored by the cluster cool gas mass fraction at $z = 0$, while the black dashed line shows the median trend. Bottom right: Marginalized distribution of distances of tracers as a function of redshift. Color indicates the bin-wise mean tracer mass, per bin per cluster. The dashed white line shows the mean virial radius of the cluster sample, and the two dashed white lines show the minimum and maximum virial radius of the sample in each redshift bin. Clusters gather most of the gas that eventually forms redshift $z=0$ cool gas around $z \sim 0.1$. (*fig:results:tracers_dist*)

</div>
<div id="div_fig2">

<img src="tmp_2503.01960/./img/tracer_history_time_development_temperature_TNG_Cluster_mean.png" alt="Fig12.1" width="50%"/><img src="tmp_2503.01960/./img/tracer_history_time_development_temperature_TNG_Cluster_2dhist_mean.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** Evolution of the temperature of tracers that contribute to $z=0$ cool gas in clusters, from TNG-Cluster. Left panel: mean temperature of gas cells associated with the cool gas tracers in each cluster as a function of redshift. Each halo of TNG-Cluster is shown as a single line, with color denoting the $z=0$ cluster mass $M_{200c}$. The black solid (dashed) line shows the mean (median) over the entire cluster sample. Right panel: distribution of temperature for gas cells associated with the selected tracers with redshift, showing the bin-wise mean over the individual distributions of all clusters. The faint dashed white line shows the mean virial temperature of the cluster sample, and the two faint dashed white lines show the minimum and maximum virial radius of the sample in each redshift bin. The two straight dashed white lines show the division between the temperature regimes at $10^{4.5}$(below which we talk of cool gas) and $10^{5.5} \rm K$(above which we talk of hot gas), respectively. The predecessor gas to redshift $z=0$ cool gas on average spends most of the time since $z = 8$ at temperatures $> 10^5  \rm K$, but below the virial temperature of the host cluster, and cooled only recently. (*fig:results:temperature_evo*)

</div>
<div id="div_fig3">

<img src="tmp_2503.01960/./img/mass_trends_standard_median_TNG300_1.png" alt="Fig1" width="100%"/>

**Figure 1. -** The fraction of each gas phase relative to the total gas mass (left) and the absolute gas mass for each phase (right) as functions of halo mass (x-axis) across all halos and clusters of TNG300 at $z=0$ . We divide gas into cool (first row, blue), warm (second row, purple), and hot (third row, red) components. The color indicates the number of halos per pixel. The black markers are the median in both gas fraction/gas mass and halo mass for mass bins of width 1 dex from $10^8$ to $10^{15}  \rm M_\odot$. The error bars denote the $\nth${16} and $\nth${84} percentile along both axes. More massive clusters host larger amounts of cool gas within their FoF halos, even though fractionally the contribution of cool and warm gas to clusters decreases with total mass. (*fig:results:mass_trends_tng300*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.01960"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\msun}{ M_{\odot}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# X-ray cavities in TNG-Cluster: a direct comparison to observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.01965-b31b1b.svg)](https://arxiv.org/abs/2503.01965)<mark>Appeared on: 2025-03-05</mark> -  _Submitted to MNRAS. The data of the TNG-Cluster simulation is now fully public and accessible at this https URL. See also science results by Chatzigiannakis+, Lehle+ and Staffehl+ on astro-ph today_

</div>
<div id="authors">

<mark>M. Prunier</mark>, <mark>A. Pillepich</mark>, J. Hlavacek-Larrondo, D. Nelson

</div>
<div id="abstract">

**Abstract:** The TNG-Cluster magnetohydrodynamic cosmological simulations, produce a diverse population of X-ray cavities in the intracluster medium (ICM) of simulated galaxy clusters. These arise from episodic, high velocity, kinetic energy injections from the central active supermassive black hole (AGN, SMBH). Here, we present the first comprehensive comparative analysis of X-ray cavities in TNG-Cluster with observational data. First, we select a volume-limited sample of 35 real clusters ( $z \leq 0.071$ , M $_\text{500c}$ = 10 $^{14-14.8}$ M $_\odot$ ) observed with the Chandra X-ray Observatory, identify 3 analogs for each in TNG-Cluster (total of 105) and generate mock Chandra images using same exposure times as their observed counterparts. We identify X-ray cavities and measure their properties in both datasets using identical techniques, ensuring a direct, apples-to-apples comparison. Our analysis reveals that both samples have a similar fraction of X-ray cavities (35-43 per cent). They exhibit comparable sizes and morphologies, although the sizes of simulated X-ray cavities still attached to the SMBH are somewhat larger in TNG-Cluster -- a scarcity at $< 10$ kpc. The area of TNG X-ray cavities increases as they rise in the ICM, consistent with the trend seen in the observational sample. The cavity powers, estimated using observational techniques, show good agreement between the two samples (10 $^{42-45}$ erg s $^{-1}$ ), suggesting that X-ray cavities in the simulation are an important heating mechanism in cluster cores. Overall, the rather simple AGN feedback model of TNG, with no model choices made to reproduce X-ray morphological features, and without cosmic rays, creates a quantitatively realistic population of X-ray cavities at cluster scales.

</div>

<div id="div_fig1">

<img src="tmp_2503.01965/./figures/A2_Pcav_Lcool_Hist_TNG_grey_other_cavz_02_logbin_stats.png" alt="Fig13" width="100%"/>

**Figure 13. -** X-ray cavity power (P$_\text{cav}$) vs. cooling luminosity (L$_\text{cool}$) for clusters with identified X-ray cavities in the TNG-Cluster sample (blue) and our observational sample (grey). Each point represents the power inferred for a single X-ray cavity; pairs are not summed up. The pink, black, and blue dashed lines represent equality between cooling and heating, assuming energy inputs of p$_\text{th}$V, 4p$_\text{th}$V, and 16p$_\text{th}$V per X-ray cavity, respectively.
    The side histograms display the distribution of P$_\text{cav}$ and L$_\text{cool}$, with 50th (dashed lines) and 16th-84th percentiles quoted in the legend. The light-grey triangles show the total X-ray cavity power in clusters of other observational studies with $z<0.2$ ([Rafferty, et. al 2006](https://ui.adsabs.harvard.edu/abs/2006ApJ...652..216R), [Dursi and Pfrommer 2008](https://ui.adsabs.harvard.edu/abs/2008ApJ...677..993D), [Hlavacek-Larrondo, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.421.1360H), [Pandge, et. al 2013](https://ui.adsabs.harvard.edu/abs/2013Ap&SS.345..183P), [Sonkamble, et. al 2015](https://ui.adsabs.harvard.edu/abs/2015Ap&SS.359...61S), [Vagshette, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.466.2054V), [Kadam, et. al 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.4113K), [Pandge, et. al 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...870...62P), [Vagshette, Naik and Patil 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.485.1981V), [ and Liu 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.492.3156L), [ and Prasow-Émond 2020](https://ui.adsabs.harvard.edu/abs/2020AJ....160..103P), [Kyadampure, Vagshette and Patil 2021](https://ui.adsabs.harvard.edu/abs/2021SerAJ.202...17K)) . The inset histogram shows the enthalpy distribution of the two X-ray cavity populations. In both samples, the power and enthalpy of X-ray cavities, and the cooling luminosity distributions are generally consistent (similar median and width). Importantly, TNG-Cluster and observed X-ray cavities exhibit well-consistent scaling relations, in terms of slope, normalization, and scatter. (*fig:Pcav_Lcool*)

</div>
<div id="div_fig2">

<img src="tmp_2503.01965/./figures/A2_Area_Distance_log_binned_final.png" alt="Fig12" width="100%"/>

**Figure 12. -** Comparison of X-ray cavity areas and distances from the central SMBH in the observed Prunier+2025 and TNG-Cluster matched samples. _ Main left panel:_ X-ray cavity areas vs. distance from the central SMBH (projected on the plane of the sky), with the Pearson coefficient quantifying the area-distance correlation for each sample. Blue and grey lines represent the best logarithmic fits. Red squared points denote the _attached_ X-ray cavities - i.e. for which size is equivalent to their distance from the central SMBH - in contrast to detached ones that are rising in the ICM. Histograms in the subpanels show the distribution of distances and areas, with 50th (dashed lines) and 16th-84th percentiles.
  _Right panels_, top and bottom histograms: distribution of X-ray cavity areas for attached and detached cavities, with 50th (dashed lines) and 16th-84th percentiles quoted in the legend.
  Both the observed and TNG-Cluster X-ray cavities show similar slopes and scatter in the area-distance relation, where X-ray cavities expand as they rise buoyantly from the central SMBH. We find a lack of small ($\lesssim$ 100 kpc$^2$) _ attached_ X-ray cavities in the TNG-Cluster sample compared to the observed one. (*fig:area_distance*)

</div>
<div id="div_fig3">

<img src="" alt="Fig9.1" width="50%"/><img src="" alt="Fig9.2" width="50%"/>

**Figure 9. -** Demographics of clusters with and without identified X-ray cavities in the observational and TNG-Cluster samples. _Left:_ per centage of clusters with identified X-ray cavities, with colored bars for TNG-Cluster and grey for the observational sample. _Right:_ Distribution of clusters with one, two, three, or more X-ray cavities, with black-edged (grey-edged) slices for TNG-Cluster (for observations). Both samples have a comparable detection rate. Clusters from the observational sample show a majority of pairs, while TNG-Cluster systems most often host a single X-ray cavity. (*fig:demographics*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.01965"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mvir}{{\rm M}_{\rm 500c}}$
$\newcommand{\rvir}{r_{\rm 500c}}$
$\newcommand{\mtc}{{\rm M}_{\rm 200c}}$
$\newcommand{\rtc}{r_{\rm 200c}}$
$\newcommand{\msun}{\rm{M}_\odot}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# What Drives Cluster Cool-Core Transformations?\\A Population Level Analysis of TNG-Cluster

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.01969-b31b1b.svg)](https://arxiv.org/abs/2503.01969)<mark>Appeared on: 2025-03-05</mark> -  _The data of the TNG-Cluster simulation is now fully public and accessible at this https URL. See also science results by Chatzigiannakis+, Prunier+ and Staffehl+ on astro-ph today. Submitted to A&A_

</div>
<div id="authors">

K. Lehle, D. Nelson, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** In this study, we examine the frequency and physical drivers of transformations from cool-core (CC) into non-cool-core (NCC) clusters, and vice versa, in a sample of 352 massive galaxy clusters ( $\mvir = 10^{14-15.3} \msun$ ) from the TNG-Cluster magnetohydrodynamical cosmological simulation of galaxies. By identifying transformations based on the evolution of central entropy and focusing on $z\lesssim2.5$ , we find that clusters frequently undergo such events, depending on their diverse assembly and supermassive black hole (SMBH) histories. On average, clusters experience two to three transformations. Transformations can occur in both directions and can be temporary, but those to higher entropy cores, i.e. in the qualitative direction from CC to NCC states, are the overwhelming majority. CC phases are also shorter than NCC phases, and thus overall the TNG-Cluster population forms with low-entropy cores and moves towards NCC states as cosmic time progresses. We study the role that mergers play in driving transformations, and find that mergers within $\sim 1$ Gyr prior to a transformation towards higher (but not lower) entropy cores occur statistically more often than in a random control sample. Most importantly, we find examples of mergers associated with cool-cores disruption regardless of their mass ratio or angular momentum. However, past merger activity is not a good predictor for $z=0$ CC status, at least based on core entropy, even though clusters undergoing more major and minor mergers eventually have the highest core entropy values at $z=0$ . We therefore consider the interplay between AGN feedback and evolving cluster core thermodynamics. We find that core transformations are accompanied by an increase in AGN activity, whereby frequent and repeated (kinetic) energy injections from the central SMBHs can produce a collective, long-term impact on central entropy, ultimately heating cluster cores. Whether such fast-paced periods of AGN activity are triggered by mergers is plausible, but not necessary.

</div>

<div id="div_fig1">

<img src="tmp_2503.01969/./figures/Halo_0.png" alt="Fig13" width="100%"/>

**Figure 13. -** Representative time evolution of the central cooling state of a massive galaxy cluster. In particular, the central entropy of the most massive $z=0$ halo in TNG-Cluster is given by the circular symbols (color indicating relaxedness, in terms of the ratio of kinetic to thermal energy within $\rvir$). Correspondingly, the light gray line shows the smoothed time evolution of $K_0$, while the dark gray curve shows its time derivative. Local extrema of the derivative are indicated by crosses, with larger orange crosses marking significant core transformation events (see text). The blue markers indicate the timescales or duration of the two core transformations (see text for a definition). This halo undergoes a core transformation between $z \sim 0.3-0.5$ wherein the core evolves towards a more cool-core (CC) state. In this case the transformation is temporary and lasts $\sim 2-3$ Gyr, after which the central entropy returns to its previous level. This type of behavior is one of several archetypes (see text for discussion). The pink stars mark the start of the three most massive merger events in the history of this cluster. The size of the markers indicate the mass ratio of the merging event. We mark the formation time of the cluster with the silver vertical, dashed line. Note that throughout the paper, time evolves from right to left in all plots that depict a time evolution. (*fig:K0vsTime*)

</div>
<div id="div_fig2">

<img src="tmp_2503.01969/./figures/SMBHfdbck/SMBH_1to3_new.png" alt="Fig19" width="100%"/>

**Figure 19. -** Evolution of the thermodynamical cluster core properties, as well as AGN feedback activity, as a function of time of an example cluster, the most massive cluster of TNG300 for which we have high temporal cadence output. We show a single halo, followed using the high time-resolution subbox (see text). We focus on a $\sim 2$ Gyr period of time centered on $z \simeq 0.1$ and the last transformation, and last merger, of this cluster. From top to bottom the panels show: the mean central entropy measured within 10 kpc (purple) and the 5th (blue) and 95th (orange) percentiles; the 95th percentile of the outflow velocity within 10 kpc around the central black hole (black); the instantaneous accretion rate of the central black hole (black); the gas mass within 30 kpc split into hot ($T\geq 10^{5.5}$ K, green), cold ($T< 10^{5.5}$ K, blue), and star forming phases (teal); the bulk outflow and inflow velocities within $0.5$\rvir$$(red and blue, respectively) and the 5th percentile of the inflow velocity (dark blue) and the 95th percentile of the outflow velocity (dark red) also measured within $0.5$\rvir$$. The vertical black dashed lines indicate the transformations, the light gray dash-dotted lines indicate the start and end of a transformation, and the dark gray dotted lines show the time of the closest associated merger (all lines are labeled in the first two rows). Repeated AGN feedback episodes are visible as short time-scale peaks in outflow velocity and central entropy that collectively help to transform this cluster into a NCC state. (*fig:timeseries*)

</div>
<div id="div_fig3">

<img src="tmp_2503.01969/./figures/SMBHfdbck/SMBH_6p5to9p5_new.png" alt="Fig20" width="100%"/>

**Figure 20. -** As in Fig. \ref{fig:timeseries}, showing the same five quantities and tracking the same cluster from TNG300 through time but here focusing on high redshift, i.e. showing details of the first two transformations to higher core entropy. These occur at $z \simeq 1.2$ and $z \simeq 0.95$, and are marked with the vertical black dotted lines. The merger at $t_{\rm lb} \simeq 8.5$ Gyr causes strong bulk motions in the inner ICM, enhancing the episodic AGN accretion and feedback, that collectively transform and remove cool gas from the cluster core. (*fig:timeseries2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.01969"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\neiii}{[Ne {\footnotesize III}]}$
$\newcommand{\oiii}{[O {\footnotesize III}]}$
$\newcommand{\oii}{[O {\footnotesize II}]}$
$\newcommand{\nii}{[N {\footnotesize II}]}$
$\newcommand{\sii}{[S {\footnotesize II}]}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\myr}{M_{\odot} yr^{-1}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\zsun}{Z_{\odot}}$
$\newcommand{\mstar}{M_{\star}}$
$\newcommand{\mgas}{M_{\rm gas}}$
$\newcommand{\mdust}{M_{\rm dust}}$
$\newcommand{\mdyn}{M_{\rm dyn}}$
$\newcommand{\kkmspc}{K~km~s^{-1}~pc^2}$
$\newcommand{\jykms}{Jy~km~s^{-1}}$
$\newcommand{\ci}{[C {\footnotesize I}]}$
$\newcommand{\hi}{H {\footnotesize I}}$
$\newcommand{\hii}{H {\footnotesize II}}$
$\newcommand{\co}{CO}$
$\newcommand{\citwo}{[C {\footnotesize I}](^3P_2 -  ^{3}P_1)}$
$\newcommand{\coseven}{CO (7-6)}$
$\newcommand{\lcitwo}{L_{\mathrm{[C \scriptscriptstyle{I}\scriptstyle{]}}^3P_2 -  ^3P_1}}$
$\newcommand{\lciplus}{L_{\rm[CII]}}$
$\newcommand{\lprimecitwo}{L'_{\mathrm{[C \scriptscriptstyle{I}\scriptstyle{]}}^3P_2 -  ^3P_1}}$
$\newcommand{\lprimecoseven}{L'_{\rm CO(7-6)}}$
$\newcommand{\lir}{L_{\rm IR}}$
$\newcommand{\lfir}{L_{\rm FIR}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\tdust}{T_{\rm dust}}$
$\newcommand{\subl}{sub-L^{\star}}$
$\newcommand{\cii}{[C {\footnotesize II}]}$
$\newcommand{\jwst}{JWST}$
$\newcommand{\hst}{HST}$
$\newcommand{\um}{\mum}$
$\newcommand{\cgs}{\rm erg cm^{-2} s^{-1} Å^{-1}}$
$\newcommand{\sigmastar}{\sigma_\star}$
$\newcommand{\mgii}{Mg {\sc ii}}$
$\newcommand{\mgi}{Mg {\sc i}}$
$\newcommand{\feii}{Fe {\sc ii}}$
$\newcommand{\mout}{M_{\rm out}}$
$\newcommand{\mdotout}{\dot{M}_{\rm out}}$
$\newcommand$
$\newcommand$
$\newcommand{\brenda}[1]{\textcolor{orange}{[#1]}}$
$\newcommand{\snhope}{NS\_274}$
$\newcommand{\rubies}{RUBIES-UDS-QG-z7}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# Gas outflows in two recently quenched galaxies at $z=4$ and $7$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.01990-b31b1b.svg)](https://arxiv.org/abs/2503.01990)<mark>Appeared on: 2025-03-05</mark> -  _13 pages, 7 figures + Appendix. Submitted to A&A on Jan 27, 2025. See Ito et al. (2025) on arXiv today for another result from the JWST "DeepDive" program_

</div>
<div id="authors">

F. Valentino, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** Outflows are a key element in the baryon cycle of galaxies, impacting their evolution by extracting gas, momentum, energy, and then injecting them into the surrounding medium.  The properties of gas outflows provide a fundamental test for our models of how galaxies transition from a phase of active star formation to quiescence. Here we report the detection of outflowing gas signatures in two recently quenched, massive ( $M_\star \sim 10^{10.2} M_\odot$ ) galaxies at $z=4.106$ ( $\snhope$ ) and $z=7.276$ ( $\rubies$ ) observed at rest-frame ultra-violet (UV) to near-infrared wavelengths with JWST/NIRSpec. The outflows are traced by blue-shifted magnesium (MgII) absorption lines, and in the case of the $z=4.1 $ system, also by  iron (FeII) and sodium (NaI) features. Together, these transitions broadly trace the chemically enriched neutral phase of the gaseous medium. The rest-frame optical spectra of the two sources are similar to those of local post-starburst galaxies, showing deep Balmer stellar features, relatively low $D_{\rm n}4000$ index, and minimal ongoing star formation on 10 Myr timescales as traced by the lack of bright nebular and recombination emission lines, also suggesting the absence of a strong and radiatively efficient AGN activity. The galaxies' star formation histories are consistent with a recent and abrupt quenching of star formation, which continued at rates of $\sim15$ $\myr$ averaged over the last 100 Myr of their life. In the case of $\snhope$ , dedicated millimeter observations allow us to strongly constrain also the dust obscured SFR to $<12$ $\myr$ , unambiguously confirming its quiescence. Under simple geometrical assumptions, we derive mass loading factors $\eta=\dot{M}_{\rm out}/\mathrm{SFR_{100 Myr}}\lesssim1$ and $\sim50$ for the $z=4.1$ and $z=7.3$ systems, respectively, and similarly different  energy carried by the outflows ( $\dot{E}_{\rm out} \lesssim5\times10^{40}$ and $\sim2\times10^{42} \mathrm{erg} \mathrm{s}^{-1}$ ). Supernovae feedback can account for the mass and energy of the outflow in $\snhope$ . However, the low mass loading factor and average gas velocity ( $\sim180$ $\kms$ , lower than the stellar velocity dispersion) suggest that the observed outflow is unlikely to be the primary reason behind the quenching of $\snhope$ , but it might represent a relic of the star formation process winding down. Star formation-related processes seem to be also insufficient to explain the extreme mass outflow rate of $\rubies$ , which would require an additional ejective mechanism such as an undetected AGN.  Finally, the average outflow velocities per unit stellar mass, SFR, or its surface area are consistent with those of lower-redshift post-starburst galaxies, suggesting that outflows in rapidly quenched galaxies might in substance occur similarly across cosmic time. Our findings hint at the existence of a rich tapestry of galaxy quenching pathways at high redshift, and they remark the importance of large spectroscopic samples mapping different spectral features to account for the different timescales on which different mechanisms contribute to this process.

</div>

<div id="div_fig1">

<img src="tmp_2503.01990/./figure/outflow_literature_all_scaling_with_vavg.png" alt="Fig9" width="100%"/>

**Figure 9. -** Average outflow velocity as a function of the stellar mass, $\mathrm{SFR}_{\rm 100Myr}$, and surface density of $\mathrm{SFR}_{\rm 100Myr}$(corrected by the magnification factor for $\snhope$). Filled stars indicate the targets analyzed in this work. The colored filled circles represent the homogenized literature compilation of $\mgii$ outflows across redshifts by $\ci$te[][and references therein]{davis_2023}. The results form stacking of post-starburst galaxies at $z\sim1.2$ from $\ci$te{maltby_2019} are shown with a red pentagon.
    The location of two $z\sim3$ recently quenched sources in $\ci$te{man_2021} is marked by grey squares. (*fig:literature*)

</div>
<div id="div_fig2">

<img src="tmp_2503.01990/./figure/paper_sfh_postsb.png" alt="Fig2" width="100%"/>

**Figure 2. -** Star formation histories of our targets. In orange and blue, we show the best-fit double power-law SFHs of $\rubies$ and $\snhope$ and their uncertainties, respectively. The age of the Universe is truncated at the observed redshift of $\snhope$. The orange segment marks the observed redshift of $\rubies$. (*fig:sfh*)

</div>
<div id="div_fig3">

<img src="tmp_2503.01990/./figure/paper_psb_combined_spectra_fig1.png" alt="Fig6" width="100%"/>

**Figure 6. -** Spectra, photometry, and SED modeling of $\snhope$ at $z=4.1061\pm0.0003$. _Top:_ Here we indicate the photometry (blue squares), JWST/NIRSpec medium resolution spectra (G140M+G235M, black line), and their uncertainties (gray line) that we jointly modeled with Bagpipes (red line) in the observed frame. The normalized residuals of the modeling are shown in the lower inset. For reference, we also show the PRISM spectrum (blue).
    _Center:_ rest-frame UV spectrum covered by the G140M grating. _Bottom:_ rest-frame optical spectrum covered by the G235M grating. We mark in red the expected location of emission and absorption lines at the derived systemic redshift $z_{\rm spec}=4.1061$, as labeled. The flux densities in the bottom panels have been rescaled by $(1+z)$ to conserve the energy. (*fig:sed_snhope*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.01990"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

349  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
